In [1]:
!pip install yt-dlp opencv-python


In [ ]:
!pip install ultralytics supervision opencv-python numpy pandas openpyxl tqdm ipywidgets -q


In [5]:
pip install pandas openpyxl


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
pd.read_csv("LIVE_REPORT.csv").to_excel("TRAFFIC_DASHBOARD.xlsx", index=False)


In [4]:
!pip install openpyxl


In [3]:
import sys
!{sys.executable} -m pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   --------------------

#!ffmpeg -y -i traffic.mp4 -c:v libx264 -pix_fmt yuv420p -movflags +faststart traffic_cv.mp4


In [3]:
VIDEO_PATH = "traffic.mp4"

In [1]:
# ========================================================
# TRAFFICSENSE AI – FINAL 100% WORKING VERSION
# Road-shaped Traffic Heatmap (Exam + Demo Ready)
# ========================================================

import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict

# ----------------------------
# CONFIG (LOCKED & TESTED)
# ----------------------------
VIDEO_PATH = "traffic.mp4"
OUTPUT_VIDEO = "traffic_output.mp4"
CSV_REPORT = "traffic_report.csv"
FINAL_HEATMAP_IMG = "FINAL_ROAD_HEATMAP.png"

VEHICLE_CLASSES = [2, 3, 5, 7]  # car, bike, bus, truck
CLASS_NAMES = {2: "car", 3: "bike", 5: "bus", 7: "truck"}

FPS_FALLBACK = 30
PIXEL_TO_METER = 0.05
SPEED_LIMIT = 40
JAM_THRESHOLD = 50

# 🔥 HEATMAP CORE (DO NOT CHANGE)
HEAT_RADIUS = 35
HEAT_VALUE = 220
GAUSSIAN_KERNEL = 31
DECAY = 0.995
HEATMAP_ALPHA = 0.8

# ----------------------------
# LOAD MODEL
# ----------------------------
print("🚀 Starting TrafficSense AI")
model = YOLO("yolo11n.pt")

# ----------------------------
# VIDEO SETUP
# ----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "❌ Video not found"

W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) or FPS_FALLBACK

writer = cv2.VideoWriter(
    OUTPUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W, H)
)

# ----------------------------
# TRACKING & TOOLS
# ----------------------------
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator()

# ----------------------------
# STORAGE
# ----------------------------
heatmap = np.zeros((H, W), dtype=np.float32)
previous_y = {}
previous_frame = {}
counted_ids = set()
rows = []

# ----------------------------
# PROCESS VIDEO
# ----------------------------
frame_id = 0
print("🔄 Processing frames...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # YOLO Detection + Tracking
    results = model.track(
        frame,
        persist=True,
        conf=0.35,
        classes=VEHICLE_CLASSES,
        verbose=False
    )[0]

    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    # ----------------------------
    # DETECTION LOOP
    # ----------------------------
    for i in range(len(detections)):
        tid = detections.tracker_id[i]
        cls = detections.class_id[i]

        if cls not in CLASS_NAMES:
            continue

        x1, y1, x2, y2 = detections.xyxy[i]
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

        # ✅ ROAD-SHAPED HEAT ADDITION (CORE FIX)
        cv2.circle(
            heatmap,
            (cx, cy),
            HEAT_RADIUS,
            HEAT_VALUE,
            -1
        )

        # Speed calculation
        if tid not in previous_y:
            previous_y[tid] = cy
            previous_frame[tid] = frame_id
            continue

        dy = abs(cy - previous_y[tid]) * PIXEL_TO_METER
        dt = (frame_id - previous_frame[tid]) / fps
        speed = (dy / dt) * 3.6 if dt > 0 else 0

        if tid not in counted_ids:
            counted_ids.add(tid)
            rows.append({
                "frame": frame_id,
                "vehicle": CLASS_NAMES[cls],
                "speed_kmh": round(speed, 2),
                "violation": speed > SPEED_LIMIT
            })

        previous_y[tid] = cy
        previous_frame[tid] = frame_id

    # ----------------------------
    # HEATMAP PROCESSING (FINAL)
    # ----------------------------
    heatmap_blur = cv2.GaussianBlur(
        heatmap,
        (GAUSSIAN_KERNEL, GAUSSIAN_KERNEL),
        0
    )

    max_val = np.max(heatmap_blur)
    heatmap_norm = (
        (heatmap_blur / max_val) * 255
    ).astype(np.uint8) if max_val > 0 else heatmap_blur.astype(np.uint8)

    heatmap_color = cv2.applyColorMap(
        heatmap_norm,
        cv2.COLORMAP_JET
    )

    overlay = cv2.addWeighted(
        frame, 0.3,
        heatmap_color, HEATMAP_ALPHA,
        0
    )

    # VERY SLOW DECAY (CRITICAL)
    heatmap *= DECAY

    # ----------------------------
    # DISPLAY
    # ----------------------------
    annotated = box_annotator.annotate(overlay.copy(), detections)

    cv2.putText(
        annotated,
        "Traffic Heatmap (Red = Congestion)",
        (30, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 255, 255),
        2
    )

    cv2.imshow("TRAFFICSENSE AI", annotated)
    writer.write(annotated)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ----------------------------
# CLEANUP
# ----------------------------
cap.release()
writer.release()
cv2.destroyAllWindows()

# ----------------------------
# SAVE REPORTS
# ----------------------------
df = pd.DataFrame(rows)
df.to_csv(CSV_REPORT, index=False)

final_blur = cv2.GaussianBlur(heatmap, (31, 31), 0)
final_norm = (final_blur / np.max(final_blur) * 255).astype(np.uint8)
final_color = cv2.applyColorMap(final_norm, cv2.COLORMAP_JET)
cv2.imwrite(FINAL_HEATMAP_IMG, final_color)

print("\n✅ DONE")
print("📁 Video :", OUTPUT_VIDEO)
print("📁 CSV   :", CSV_REPORT)
print("📁 Heatmap:", FINAL_HEATMAP_IMG)
print("🎯 Road-shaped heatmap SUCCESS")


WARNING  Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\skush\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


🚀 Starting TrafficSense AI
🔄 Processing frames...

✅ DONE
📁 Video : traffic_output.mp4
📁 CSV   : traffic_report.csv
📁 Heatmap: FINAL_ROAD_HEATMAP.png
🎯 Road-shaped heatmap SUCCESS
